Import des bibliothèques python :

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


In [2]:
#1)Importation de pandas et lecture du fichier csv 
#on charge le fichier dtb.csv dans un dataframe(structure tabulaire) pandas "df"
df = pd.read_csv("dtb.csv")
#puis on affiche son contenu 
print(df)
#on affiche la dimension (nbr de lignes et de colonnes)
print(df.shape)

#2)Définir une class Dataset personnalisé 
#hérité de torch.utils.data.Dataset avec laquelle on peut créer un jeu de données 
class DtSet(Dataset):
    #on initialise l'objet
    def __init__(self, dataframe):
        #on selectionne toutes les colonnes sauf la ddernière et on recupère leurs valeurs(tableau)
        self.x = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        #on va sélectionner seulemenet la derr colonne qui est souvent la cible 
        self.y = torch.tensor(dataframe.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    #ccette methode permet de recuperer un elem individuel à partir d'un dataset
    #elle prend un indice en entrée et renvoie la paire x,y 
    def __getitem__(self, ind):
        return self.x[ind], self.y[ind]
#creation de l'instance    
instance = DtSet(df)
#dataloader est un utilitaire pytorch qui permet de faire des batches 
#avec l'objet , la taille du lot (3 par exemple) et shuffle true permet de melanger les données
loader = torch.utils.data.DataLoader(instance, batch_size=3, shuffle=True)

print(instance.x, "\n",  instance.y)

     X   Y   Z  res
0    1   2   3    1
1    2   3   4    2
2    3   2   1    3
3    2   1   2    2
4    5   1   1    5
5    4   1   2    4
6   10   1   0   10
7   11   7   6   11
8   15   9   8   15
9   20   9   1   20
10  13   3   3   13
11   3   4   5    3
12   5  13   3    5
13  17  16  16   17
(14, 4)
tensor([[ 1.,  2.,  3.],
        [ 2.,  3.,  4.],
        [ 3.,  2.,  1.],
        [ 2.,  1.,  2.],
        [ 5.,  1.,  1.],
        [ 4.,  1.,  2.],
        [10.,  1.,  0.],
        [11.,  7.,  6.],
        [15.,  9.,  8.],
        [20.,  9.,  1.],
        [13.,  3.,  3.],
        [ 3.,  4.,  5.],
        [ 5., 13.,  3.],
        [17., 16., 16.]]) 
 tensor([ 1.,  2.,  3.,  2.,  5.,  4., 10., 11., 15., 20., 13.,  3.,  5., 17.])


Mauvaise Normalisation des données (Z - score) (creation of noise)

In [3]:


"""for X, Y in loader:
    mean = X.mean()
    std = X.std()
    X = (X - mean) / std"""





'for X, Y in loader:\n    mean = X.mean()\n    std = X.std()\n    X = (X - mean) / std'

In [4]:
# True normalisation (Z - score)
#on extrait les  données  x,y,z du dataframe 
X = df[["X", "Y", "Z"]].values #(input)
#on extrait la colonne res qui est la cible dans ce cas (output)
Y = df[["res"]].values
#on crée deux normalisateurs Z-score 
#la methode z-score va transformer chaque valeur selon x_norm= x−μ/σ 
scaler_x = StandardScaler()
scaler_y = StandardScaler()
#Transformation et conversion en tensors pytorch 
"""
fit_transform : 
    fit calcule la moyenne et l'écart-type sur les données
    transform applique la normalisation Z-score
"""
X_tensor = torch.tensor(scaler_x.fit_transform(X), dtype = torch.float32)
Y_tensor = torch.tensor(scaler_y.fit_transform(Y), dtype = torch.float32)


print(X_tensor, "\n", Y_tensor)

tensor([[-1.1402, -0.6661, -0.2348],
        [-0.9757, -0.4541,  0.0181],
        [-0.8111, -0.6661, -0.7406],
        [-0.9757, -0.8780, -0.4877],
        [-0.4820, -0.8780, -0.7406],
        [-0.6465, -0.8780, -0.4877],
        [ 0.3409, -0.8780, -0.9935],
        [ 0.5055,  0.3936,  0.5238],
        [ 1.1637,  0.8174,  1.0296],
        [ 1.9866,  0.8174, -0.7406],
        [ 0.8346, -0.4541, -0.2348],
        [-0.8111, -0.2422,  0.2709],
        [-0.4820,  1.6651, -0.2348],
        [ 1.4929,  2.3009,  3.0526]]) 
 tensor([[-1.1402],
        [-0.9757],
        [-0.8111],
        [-0.9757],
        [-0.4820],
        [-0.6465],
        [ 0.3409],
        [ 0.5055],
        [ 1.1637],
        [ 1.9866],
        [ 0.8346],
        [-0.8111],
        [-0.4820],
        [ 1.4929]])


In [5]:
class LinRegModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 3),
        self.linear = nn.Linear(3, 6),
        self.linear = nn.Linear(6, 6),
        self.linear = nn.Linear(6, 9),
        self.linear = nn.Linear(9, 12),
        self.linear = nn.Linear(12, 12),
        self.linear = nn.Linear(12, 9),
        self.linear = nn.Linear(9, 9),
        self.linear = nn.Linear(9, 7),
        self.linear = nn.Linear(7, 5),
        self.linear = nn.Linear(5, 3),
        self.linear = nn.Linear(3, 1)


    def forward(self, x):
        return self.linear(x)

In [6]:
in_features = 3 
out_features = 1
# régression linéaire simple
model = LinRegModel(in_features, out_features)

In [7]:
# Mean Squared Error comme fonction de perte
criterion = nn.MSELoss()

# Stochastic Gradient Descent (SGD) avec un learning rate de 0.1.
""" 
model.parameters() récupère tous les poids (et biais) entraînables
"""
optimizer = optim.SGD(model.parameters(), lr = 0.1)


In [8]:
nb_epochs = 100 # nombre d’itérations

for i in range(nb_epochs):
    # Propagation avant
    """
    calcule de la sortie prédite en appliquant la régression linéaire.
    """
    outputs = model(X_tensor) 

    # fonction de perte (loss)
    # Compare les sorties prédites outputs avec les vraies valeurs Y_tensor.
    loss = criterion(outputs, Y_tensor) 

    # Remise à zéro des gradients précédents pour éviter l'accumulation des gradients d’itérations précédentes.

    optimizer.zero_grad()
    loss.backward()
    # Mise à jour des paramètres du modèle selon les gradients et la règle d’optimisation
    optimizer.step()

    print(f'Epoch [{i+ 1}], Loss: {loss.item():.9f}')



Epoch [1], Loss: 0.540813744
Epoch [2], Loss: 0.331718385
Epoch [3], Loss: 0.227787837
Epoch [4], Loss: 0.166408136
Epoch [5], Loss: 0.125341102
Epoch [6], Loss: 0.095822297
Epoch [7], Loss: 0.073818192
Epoch [8], Loss: 0.057119720
Epoch [9], Loss: 0.044331007
Epoch [10], Loss: 0.034485806
Epoch [11], Loss: 0.026881406
Epoch [12], Loss: 0.020993546
Epoch [13], Loss: 0.016425831
Epoch [14], Loss: 0.012876294
Epoch [15], Loss: 0.010113747
Epoch [16], Loss: 0.007960592
Epoch [17], Loss: 0.006280036
Epoch [18], Loss: 0.004966487
Epoch [19], Loss: 0.003938301
Epoch [20], Loss: 0.003132242
Epoch [21], Loss: 0.002499285
Epoch [22], Loss: 0.002001361
Epoch [23], Loss: 0.001608891
Epoch [24], Loss: 0.001298877
Epoch [25], Loss: 0.001053403
Epoch [26], Loss: 0.000858509
Epoch [27], Loss: 0.000703318
Epoch [28], Loss: 0.000579338
Epoch [29], Loss: 0.000479934
Epoch [30], Loss: 0.000399918
Epoch [31], Loss: 0.000335233
Epoch [32], Loss: 0.000282700
Epoch [33], Loss: 0.000239823
Epoch [34], Loss: 0

In [9]:

new_x = np.array([[5223, 1, 2025]])

scalar_test = StandardScaler()

new_x_tensor = torch.tensor((new_x), dtype = torch.float32)

model.eval() # Évite les effets indésirables si ton modèle avait des couches comme Dropout ou BatchNorm.

with torch.no_grad(): # Évite de stocker les gradients (inutiles ici), donc plus rapide et moins de mémoire
    prediction_norm = model(new_x_tensor)

prediction_denorm = prediction_norm.item() 

print(f"Predicted for X = {new_x}: {(prediction_denorm)}")

Predicted for X = [[5223    1 2025]]: 5220.447265625
